<a href="https://colab.research.google.com/github/qiumic000/fastai/blob/master/bearwidget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Bear Classifier!**
This classifier takes in bear images and determines the probability of the image being a grizzly bear, a black bear, or a teddy bear. Enjoy :)

In [2]:
from fastai2.vision.all import *
from fastai2.vision.widgets import *

ModuleNotFoundError: ignored

In [1]:
path = Path()
learn_inf = load_learner(path/'export.pkl')
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description = "Classify")
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
  
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

NameError: ignored